# Trabalho prático de Análise Inteligente em Sistemas de "Big Data"

Importar bibliotecas

In [12]:
import pandas as pd
import time
from pymongo import MongoClient
from pyspark.sql import SparkSession
from unidecode import unidecode

# Função para remover acentos de uma string
def remover_acentos(texto):
    return unidecode(texto)

Juntar os datasets e medir o tempo de execução com Python

In [58]:
# Measure exec time: Start point
start_time = time.time()

# Read the data from the CSV files
ds1 = pd.read_excel('dataset_input/efeito_estufa.xlsx')
ds2 = pd.read_excel('dataset_input/emissoes_gases.xlsx')
ds3 = pd.read_excel('dataset_input/intensidade_carbonica.xlsx')
ds4 = pd.read_excel('dataset_input/PIBcrescimento.xlsx')
ds5 = pd.read_excel('dataset_input/PIBdespesas.xlsx')
ds6 = pd.read_excel('dataset_input/PIBproducao.xlsx')
ds7 = pd.read_excel('dataset_input/PIBrendimento.xlsx')

ds1 = ds1.add_suffix('_efeito_estufa')
ds1.rename(columns={'Anos_efeito_estufa': 'anos'}, inplace=True)

ds2.rename(columns={'Anos': 'anos'}, inplace=True)

ds3 = ds3.add_suffix('_intensidade_carbonica')
ds3.rename(columns={'Anos_intensidade_carbonica': 'anos'}, inplace=True)

ds4 = ds4.add_suffix('_PIBcrescimento')
ds4.rename(columns={'Anos_PIBcrescimento': 'anos'}, inplace=True)

ds5 = ds5.add_suffix('_PIBdespesas')
ds5.rename(columns={'Anos_PIBdespesas': 'anos'}, inplace=True)

ds6 = ds6.add_suffix('_PIBproducao')
ds6.rename(columns={'Anos_PIBproducao': 'anos'}, inplace=True)

ds7 = ds7.add_suffix('_PIBrendimento')
ds7.rename(columns={'Anos_PIBrendimento': 'anos'}, inplace=True)

print(ds1.columns)
#Merge the data
dsAmbiente = pd.merge(pd.merge(ds1, ds2, on='anos'), ds3, on='anos')
dsPIB = pd.merge(pd.merge(ds4, ds5, on='anos'), pd.merge(ds6, ds7, on='anos'), on='anos')

# verify joined datasets
dsPIB.dropna(inplace=True)
# drop repeated columns total, total_x and total_y are all the same values
#dsPIB.drop(columns=['Total_x', 'Total_y'], inplace=True)

dsAmbiente.dropna(inplace=True)

ds = pd.merge(dsAmbiente, dsPIB, on='anos')

ds.columns = map(remover_acentos, ds.columns.str.replace('das ', '').str.replace('de ', '').str.replace('e ', '').str.replace(',', '').str.replace(' ', '_').str.lower())

ds.to_csv('dataset_output/output.csv', index=False)

# Measure exec time: End point
end_time = time.time()

# Calculate execution time and show in stdout
exec_time = end_time - start_time
print(f"Execution time: {round(exec_time, 3)} seconds")

Index(['anos', 'Total_efeito_estufa',
       'Total das atividades economicas_efeito_estufa',
       'Total das famílias_efeito_estufa'],
      dtype='object')
Execution time: 0.221 seconds


Utilizar o PySpark para medição do tempo de execução, com conversão do dataset para formato Parquet (*Nota: Aparente erro na função write*)

In [54]:
session = SparkSession.builder.appName("CSV to Parquet").getOrCreate()

df_spark = session.read.csv("dataset_output/output.csv", header=True, inferSchema=True)

df_spark.write.parquet("dataset_output/output.parquet") # Write pode não estar funcional

session.stop()

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/c:/Users/Work/Documents/Aulas2023_24/sem2/BD/BigData/dataset_output/output.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

Introdução do ficheiro ~~Parquet~~ (*not working*) CSV para uma base de dados (MongoDB)

In [6]:
# Connect to MongoDB

from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["BigData"]
collection = db["data"]

df = pd.read_csv("dataset_output/output.csv")
dictionary = df.to_dict(orient="records")

collection.insert_many(dictionary)

InsertManyResult([ObjectId('6617bc99b8a60fe8dac962c6'), ObjectId('6617bc99b8a60fe8dac962c7'), ObjectId('6617bc99b8a60fe8dac962c8'), ObjectId('6617bc99b8a60fe8dac962c9'), ObjectId('6617bc99b8a60fe8dac962ca'), ObjectId('6617bc99b8a60fe8dac962cb'), ObjectId('6617bc99b8a60fe8dac962cc'), ObjectId('6617bc99b8a60fe8dac962cd'), ObjectId('6617bc99b8a60fe8dac962ce'), ObjectId('6617bc99b8a60fe8dac962cf'), ObjectId('6617bc99b8a60fe8dac962d0'), ObjectId('6617bc99b8a60fe8dac962d1'), ObjectId('6617bc99b8a60fe8dac962d2'), ObjectId('6617bc99b8a60fe8dac962d3'), ObjectId('6617bc99b8a60fe8dac962d4'), ObjectId('6617bc99b8a60fe8dac962d5'), ObjectId('6617bc99b8a60fe8dac962d6'), ObjectId('6617bc99b8a60fe8dac962d7'), ObjectId('6617bc99b8a60fe8dac962d8'), ObjectId('6617bc99b8a60fe8dac962d9'), ObjectId('6617bc99b8a60fe8dac962da'), ObjectId('6617bc99b8a60fe8dac962db'), ObjectId('6617bc99b8a60fe8dac962dc'), ObjectId('6617bc99b8a60fe8dac962dd'), ObjectId('6617bc99b8a60fe8dac962de'), ObjectId('6617bc99b8a60fe8dac962